In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

print("IMPORT")
import tensorflow as tf #tensorflow
print(tf.__version__)
import numpy as np #numpy > save loss .. 
from collections import OrderedDict #layer ..
import os, random #dir, random..
import pickle #save & load

# cifar 10 
#wget http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
#tar xzf cifar-10-python.tar.gz

#resnet
#https://arxiv.org/abs/1512.03385

IMPORT
1.0.1


In [2]:
print("PATHS")
#path
DATA_PATH = 'cifar10/cifar-10-batches-py/'
CHECKPOINT = 'simple_res_net.ckpt'
SAVE_PATH = 'checkpoint/simple_res_net/'
BOARD_PATH = './tensorboard/board_res_net'
print("VARIABLES")
#parameters
INPUT_SIDE = 32
INPUT_SIZE = INPUT_SIDE * INPUT_SIDE
N_CHANNEL = 3
BATCH_SIZE = 64
EPOCHS = 100
LR = 0.01
LR_DECAY_RATE = 0.5

LIST_CLASS=['airplane', 'automobile', 'birds', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
N_CLASSES = len(LIST_CLASS)

#train&test batch
train_file=['data_batch_1','data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
test_file=['cifar10/cifar-10-batches-py/test_batch']

#input & output 
x = tf.placeholder("float", [BATCH_SIZE, INPUT_SIZE * N_CHANNEL]) #batch x image size
y = tf.placeholder("float", [BATCH_SIZE, N_CLASSES]) #batch x class`


print("input image size : {}".format(INPUT_SIZE))
print("image channel : {}".format(N_CHANNEL))
print("batch size : {}".format(BATCH_SIZE))
print("num of class : {}".format(N_CLASSES))
print("training epochs : {}".format(EPOCHS))
print("learning rate : {}".format(LR))
print("learning decay rate : {}".format(LR_DECAY_RATE))

PATHS
VARIABLES
input image size : 1024
image channel : 3
batch size : 64
num of class : 10
training epochs : 100
learning rate : 0.01
learning decay rate : 0.5


In [3]:
#NETWORK PARAMETERS

print("NETWORK PARAMETERS")

stddev = 0.1

weights = {
     'conv' : tf.Variable(tf.random_normal([3, 3, N_CHANNEL, 16], stddev=stddev), name='conv'),
    'conv1_1x1' : tf.Variable(tf.random_normal([1, 1, 16, 4], stddev=stddev)),
    'conv1_3x3' : tf.Variable(tf.random_normal([3, 3, 4, 4], stddev=stddev)),
    'conv1_1x1_16' : tf.Variable(tf.random_normal([1, 1, 4, 16], stddev=stddev)),
    'conv2_1x1' : tf.Variable(tf.random_normal([1, 1, 16, 4], stddev=stddev)),
    'conv2_3x3' : tf.Variable(tf.random_normal([3, 3, 4, 4], stddev=stddev)),
    'conv2_1x1_16' : tf.Variable(tf.random_normal([1, 1, 4, 16], stddev=stddev)),
    
    # conv 16 + conv2 16 = 32filters 
    
    'conv3_1x1' : tf.Variable(tf.random_normal([1, 1, 32, 8], stddev=stddev)),
    'conv3_3x3' : tf.Variable(tf.random_normal([3, 3, 8, 8], stddev=stddev)),
    'conv3_1x1_32' : tf.Variable(tf.random_normal([1, 1, 8, 32], stddev=stddev)),
    'conv4_1x1' : tf.Variable(tf.random_normal([1, 1, 32, 8], stddev=stddev)),
    'conv4_3x3' : tf.Variable(tf.random_normal([3, 3, 8, 8], stddev=stddev)),
    'conv4_1x1_32' : tf.Variable(tf.random_normal([1, 1, 8, 32], stddev=stddev)),
    
    # conv2 지난거32 + conv4 32 = 64
    
    'conv5_1x1' : tf.Variable(tf.random_normal([1, 1, 64, 16], stddev=stddev)),
    'conv5_3x3' : tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'conv5_1x1_64' : tf.Variable(tf.random_normal([1, 1, 16, 64], stddev=stddev)),
    'conv6_1x1' : tf.Variable(tf.random_normal([1, 1, 64, 16], stddev=stddev)),
    'conv6_3x3' : tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'conv6_1x1_64' : tf.Variable(tf.random_normal([1, 1, 16, 64], stddev=stddev)),
    
    #conv4 지난거 64 + conv6 64 = 128
    
    'dense1' : tf.Variable(tf.random_normal([16*16*128, 1000], stddev=stddev)),
    'dense2' : tf.Variable(tf.random_normal([1000, N_CLASSES], stddev=stddev))    
}
biases = {
    'conv' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    'conv1_1x1' : tf.Variable(tf.random_normal([4], stddev=stddev)),
    'conv1_3x3' : tf.Variable(tf.random_normal([4], stddev=stddev)),
    'conv1_1x1_16' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    'conv2_1x1' : tf.Variable(tf.random_normal([4], stddev=stddev)),
    'conv2_3x3' :tf.Variable(tf.random_normal([4], stddev=stddev)),
    'conv2_1x1_16' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    
    'conv3_1x1' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    'conv3_3x3' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    'conv3_1x1_32' : tf.Variable(tf.random_normal([32], stddev=stddev)),
    'conv4_1x1' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    'conv4_3x3' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    'conv4_1x1_32' : tf.Variable(tf.random_normal([32], stddev=stddev)),
    
    'conv5_1x1' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    'conv5_3x3' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    'conv5_1x1_64' : tf.Variable(tf.random_normal([64], stddev=stddev)),
    'conv6_1x1' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    'conv6_3x3' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    'conv6_1x1_64' : tf.Variable(tf.random_normal([64], stddev=stddev)),
    
    'dense1' : tf.Variable(tf.random_normal([1000], stddev=stddev)),
    'dense2' : tf.Variable(tf.random_normal([N_CLASSES], stddev=stddev))
}

NETWORK PARAMETERS


In [4]:
#model
#http://laonple.blog.me/220764986252 - bottleneck 
#conv 3x3
#conv (1x1, 3x3, 1x1) - relu > 16
#conv (1x1, 3x3, 1x1) relu > 32
#conv (1x1, 3x3, 1x1) relu > 64
#avg pooling 
#fc
#softmax

print("NETWORK FUNCTION")

def SimpleResNet(img_width, img_height, img_channel, _x, _w, _b, scope='SimpleResNet'):
    network = OrderedDict() #network layers

    # X RESHAPE
    _x_r = tf.reshape(_x, shape=[-1,img_width,img_height, img_channel])
    
    with tf.variable_scope(scope):
        with tf.name_scope('conv') as scope:
            conv = tf.nn.conv2d(_x_r, _w['conv'], strides=[1, 1, 1, 1], padding='SAME')
            conv = tf.nn.bias_add(conv, _b['conv'])
            conv = tf.nn.relu(conv)
            network['conv'] = conv
        
        #16
        with tf.name_scope('conv2') as scope:
            conv1_1x1 = tf.nn.conv2d(conv, _w['conv1_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            conv1_1x1 = tf.nn.bias_add(conv1_1x1, _b['conv1_1x1'])
            conv1_1x1 = tf.nn.relu(conv1_1x1)
            conv1_3x3 = tf.nn.conv2d(conv1_1x1, _w['conv1_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            conv1_3x3 = tf.nn.bias_add(conv1_3x3, _b['conv1_3x3'])
            conv1_3x3 = tf.nn.relu(conv1_3x3)
            conv1_1x1_16 = tf.nn.conv2d(conv1_3x3, _w['conv1_1x1_16'], strides=[1, 1, 1, 1], padding='SAME')
            conv1_1x1_16 = tf.nn.bias_add(conv1_1x1_16, _b['conv1_1x1_16'])
            conv1_1x1_16 = tf.nn.relu(conv1_1x1_16)
            network['conv1_1x1_16'] = conv1_1x1_16
        
        #16
        with tf.name_scope('conv3') as scope:
            conv2_1x1 = tf.nn.conv2d(conv1_1x1_16, _w['conv2_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            conv2_1x1 = tf.nn.bias_add(conv2_1x1, _b['conv2_1x1'])
            conv2_1x1 = tf.nn.relu(conv2_1x1)
            conv2_3x3 = tf.nn.conv2d(conv2_1x1, _w['conv2_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            conv2_3x3 = tf.nn.bias_add(conv2_3x3, _b['conv2_3x3'])
            conv2_3x3 = tf.nn.relu(conv2_3x3)
            conv2_1x1_16 = tf.nn.conv2d(conv2_3x3, _w['conv2_1x1_16'], strides=[1, 1, 1, 1], padding='SAME')
            conv2_1x1_16 = tf.nn.bias_add(conv2_1x1_16, _b['conv2_1x1_16'])
            #print(conv2_1x1_16.shape)
            #32
            conv2_1x1_16 = tf.concat([conv, conv2_1x1_16], 3) 
            conv2_1x1_16 = tf.nn.relu(conv2_1x1_16)
            network['conv2_1x1_16'] = conv2_1x1_16
       
        #print(conv2_1x1_16.shape)
        #32       32 x 32 x 16+16 >>> 
        with tf.name_scope('conv4') as scope:
            conv3_1x1 = tf.nn.conv2d(conv2_1x1_16, _w['conv3_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            conv3_1x1 = tf.nn.bias_add(conv3_1x1, _b['conv3_1x1'])
            conv3_1x1 = tf.nn.relu(conv3_1x1)
            conv3_3x3 = tf.nn.conv2d(conv3_1x1, _w['conv3_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            conv3_3x3 = tf.nn.bias_add(conv3_3x3, _b['conv3_3x3'])
            conv3_3x3 = tf.nn.relu(conv3_3x3)
            conv3_1x1_32 = tf.nn.conv2d(conv3_3x3, _w['conv3_1x1_32'], strides=[1, 1, 1, 1], padding='SAME')
            conv3_1x1_32 = tf.nn.bias_add(conv3_1x1_32, _b['conv3_1x1_32'])
            conv3_1x1_32 = tf.nn.relu(conv3_1x1_32)
            network['conv3_1x1_32'] = conv3_1x1_32
        
        with tf.name_scope('conv5') as scope:
            conv4_1x1 = tf.nn.conv2d(conv3_1x1_32, _w['conv4_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            conv4_1x1 = tf.nn.bias_add(conv4_1x1, _b['conv4_1x1'])
            conv4_1x1 = tf.nn.relu(conv4_1x1)
            conv4_3x3 = tf.nn.conv2d(conv4_1x1, _w['conv4_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            conv4_3x3 = tf.nn.bias_add(conv4_3x3, _b['conv4_3x3'])
            conv4_3x3 = tf.nn.relu(conv4_3x3)
            conv4_1x1_32 = tf.nn.conv2d(conv4_3x3, _w['conv4_1x1_32'], strides=[1, 1, 1, 1], padding='SAME')
            conv4_1x1_32 = tf.nn.bias_add(conv4_1x1_32, _b['conv4_1x1_32'])

            #64
            conv4_1x1_32 = tf.concat([conv2_1x1_16, conv4_1x1_32 ], 3)
            conv4_1x1_32 = tf.nn.relu(conv4_1x1_32)
            network['conv4_1x1_32'] = conv4_1x1_32
    
        with tf.name_scope('conv6') as scope:
            conv5_1x1 = tf.nn.conv2d(conv4_1x1_32, _w['conv5_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            conv5_1x1 = tf.nn.bias_add(conv5_1x1, _b['conv5_1x1'])
            conv5_1x1 = tf.nn.relu(conv5_1x1)
            conv5_3x3 = tf.nn.conv2d(conv5_1x1, _w['conv5_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            conv5_3x3 = tf.nn.bias_add(conv5_3x3, _b['conv5_3x3'])
            conv5_3x3 = tf.nn.relu(conv5_3x3)
            conv5_1x1_64 = tf.nn.conv2d(conv5_3x3, _w['conv5_1x1_64'], strides=[1, 1, 1, 1], padding='SAME')
            conv5_1x1_64 = tf.nn.bias_add(conv5_1x1_64, _b['conv5_1x1_64'])
            conv5_1x1_64 = tf.nn.relu(conv5_1x1_64)
            network['conv5_1x1_64'] = conv5_1x1_64
        
        with tf.name_scope('conv7') as scope:
            conv6_1x1 = tf.nn.conv2d(conv5_1x1_64, _w['conv6_1x1'], strides=[1, 1, 1, 1], padding='SAME')
            conv6_1x1 = tf.nn.bias_add(conv6_1x1, _b['conv6_1x1'])
            conv6_1x1 = tf.nn.relu(conv6_1x1)
            conv6_3x3 = tf.nn.conv2d(conv6_1x1, _w['conv6_3x3'], strides=[1, 1, 1, 1], padding='SAME')
            conv6_3x3 = tf.nn.bias_add(conv6_3x3, _b['conv6_3x3'])
            conv6_3x3 = tf.nn.relu(conv6_3x3)
            conv6_1x1_64 = tf.nn.conv2d(conv6_3x3, _w['conv6_1x1_64'], strides=[1, 1, 1, 1], padding='SAME')
            conv6_1x1_64 = tf.nn.bias_add(conv6_1x1_64, _b['conv6_1x1_64'])

            #128
            conv6_1x1_64 = tf.concat([conv4_1x1_32, conv6_1x1_64],3)
            conv6_1x1_64 = tf.nn.relu(conv6_1x1_64)
            network['conv6_1x1_64'] = conv6_1x1_64
        
        with tf.name_scope('pool') as scope:
            pool = tf.nn.avg_pool(conv6_1x1_64, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            network['pool'] = pool
        
        with tf.name_scope('dense1') as scope:
            dense = tf.reshape(pool, [-1, _w['dense1'].get_shape().as_list()[0]])
            dense1 = tf.add(tf.matmul(dense, _w['dense1']), _b['dense1'])
            dense1 = tf.nn.relu(dense1)
            network['dense1'] = dense1
            
        with tf.name_scope('logit') as scope:
            logit = tf.add(tf.matmul(dense1, _w['dense2']), _b['dense2'])
            network['logit'] = logit
        
    return network

NETWORK FUNCTION


In [5]:
#save
saver = tf.train.Saver(max_to_keep=3) #최근 3개까지만 저장
save_step = 100 #save for 100 epoch
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
print ("SAVER READY")

SAVER READY


In [6]:
out = SimpleResNet(INPUT_SIDE, INPUT_SIDE, N_CHANNEL, x, weights, biases, 'SimpleResNet')
print("CHECK LAYERS")
print(x)
print(y)
for key, value in out.items():
    print (key, value)

CHECK LAYERS
Tensor("Placeholder:0", shape=(64, 3072), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 10), dtype=float32)
conv Tensor("SimpleResNet/Relu:0", shape=(64, 32, 32, 16), dtype=float32)
conv1_1x1_16 Tensor("SimpleResNet/Relu_3:0", shape=(64, 32, 32, 16), dtype=float32)
conv2_1x1_16 Tensor("SimpleResNet/Relu_6:0", shape=(64, 32, 32, 32), dtype=float32)
conv3_1x1_32 Tensor("SimpleResNet/Relu_9:0", shape=(64, 32, 32, 32), dtype=float32)
conv4_1x1_32 Tensor("SimpleResNet/Relu_12:0", shape=(64, 32, 32, 64), dtype=float32)
conv5_1x1_64 Tensor("SimpleResNet/Relu_15:0", shape=(64, 32, 32, 64), dtype=float32)
conv6_1x1_64 Tensor("SimpleResNet/Relu_18:0", shape=(64, 32, 32, 128), dtype=float32)
pool Tensor("SimpleResNet/AvgPool:0", shape=(64, 16, 16, 128), dtype=float32)
dense1 Tensor("SimpleResNet/Relu_19:0", shape=(64, 1000), dtype=float32)
logit Tensor("SimpleResNet/Add_1:0", shape=(64, 10), dtype=float32)


In [7]:
#cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out['logit']))
tf.summary.scalar("loss", cost)
print("LOSS FUNCTION")

#learning rate
global_step = tf.Variable(0, trainable=False, name="global_step")
learning_rate = tf.train.exponential_decay(learning_rate=LR,
                                           global_step=global_step,
                                           decay_steps=500,
                                           decay_rate=LR_DECAY_RATE,
                                           staircase=True,
                                           name="learning_rate")
learning_rate = tf.maximum(learning_rate, 0.0001)
tf.summary.scalar("learning_rate", learning_rate)
print("LERANING RATE : {}".format(learning_rate))

#optimizer
adam = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
sgd = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)
print("OPTIMIZER 1 : {}".format(adam))
print("OPTIMIZER 2 : {}".format(sgd))

corr = tf.equal(tf.argmax(out['logit'], 1), tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(corr, "float"))
tf.summary.scalar("accuracy", accr)

LOSS FUNCTION
LERANING RATE : Tensor("Maximum:0", shape=(), dtype=float32)
OPTIMIZER 1 : name: "Adam"
op: "NoOp"
input: "^Adam/update_conv/ApplyAdam"
input: "^Adam/update_Variable/ApplyAdam"
input: "^Adam/update_Variable_1/ApplyAdam"
input: "^Adam/update_Variable_2/ApplyAdam"
input: "^Adam/update_Variable_3/ApplyAdam"
input: "^Adam/update_Variable_4/ApplyAdam"
input: "^Adam/update_Variable_5/ApplyAdam"
input: "^Adam/update_Variable_6/ApplyAdam"
input: "^Adam/update_Variable_7/ApplyAdam"
input: "^Adam/update_Variable_8/ApplyAdam"
input: "^Adam/update_Variable_9/ApplyAdam"
input: "^Adam/update_Variable_10/ApplyAdam"
input: "^Adam/update_Variable_11/ApplyAdam"
input: "^Adam/update_Variable_12/ApplyAdam"
input: "^Adam/update_Variable_13/ApplyAdam"
input: "^Adam/update_Variable_14/ApplyAdam"
input: "^Adam/update_Variable_15/ApplyAdam"
input: "^Adam/update_Variable_16/ApplyAdam"
input: "^Adam/update_Variable_17/ApplyAdam"
input: "^Adam/update_Variable_18/ApplyAdam"
input: "^Adam/update_Varia

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [8]:
#Session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

summary_op = tf.summary.merge_all() #tensor board
summary_writer = tf.summary.FileWriter(BOARD_PATH, sess.graph) #tensor board
checkpoint = tf.train.latest_checkpoint(SAVE_PATH)
if checkpoint is not None:
    print(checkpoint)
    saver.restore(sess, checkpoint)


checkpoint/simple_res_net/simple_res_net.ckpt-0


In [9]:
#npz - python numpy 파일 포맷. > dictionary로 불러옴 (Save several arrays into a single file in uncompressed .npz format)
def load_npz(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
        return dict
    
## Make batch_iterator for npz data
def make_batch_index(_data, batch_size=128, allow_small_batch=True):
    num_images = len(_data[b'data']) #데이터 갯수
    start_idx = list(range(0, num_images, batch_size)) #각 배치별 시작 인덱스 리스트
    end_idx = list(range(batch_size, num_images + 1, batch_size))#각 배치별 끝 인덱스 
    if allow_small_batch and end_idx[-1] < num_images : #스몰배치 허용시 마지막 배치
        start_idx.append(end_idx[-1])
        end_idx.append(num_images)
    return zip(start_idx, end_idx)

#one hot encoding
def one_hot_encode(x, numclass):
    return np.eye(numclass)[x] #numclass 크기의 항등행렬 생성 

In [10]:
#training
loss_for_plot = []
acc_for_plot = []

for epoch in range(EPOCHS):
    sample_list_new = random.sample(train_file, len(train_file)) #변환할 부분 - training file shuffle
    nNumBatch = 0 #배치수
    AvgBatchCost = 0 #batch cost
    
    for index, file_name in enumerate(sample_list_new): #sample file list 돌면서 index, filename을 받아온다
        file_name = DATA_PATH + file_name #해당 파일 이름
        load_data = load_npz(file_name) #해당 파일 이름으로 불러온다. 
        batch_index = make_batch_index(load_data, BATCH_SIZE, False) #배치의 시작과 끝 인덱스를 zip으로 받아온다. 
        
        for start, end in batch_index:
            batch_xs = load_data[b'data'][start:end] #start~end까지의 x데이터(image)를 준비
            batch_ys = one_hot_encode(load_data[b'labels'][start:end], N_CLASSES) # label을 원핫인코딩으로 변환해줌. 
            #print(batch_xs.shape)
            feed_dictionary = {x : batch_xs, y : batch_ys} #feed dict에 x, y준비

            #실제 실행 -> optimizer 고를듯..
            #sess.run(adam, feed_dict = feed_dictionary)
            nNumBatch += 1
            if epoch < EPOCHS*0.5:
                _, tmp_cost = sess.run([adam, cost], feed_dict = feed_dictionary) #optimizer는 결과값중요하지 않으니 _로 받고, cost를 받음
            else:
                _, tmp_cost = sess.run([sgd, cost], feed_dict = feed_dictionary) 
            
            AvgBatchCost += tmp_cost
            
        #매 배치가 끝난 후 
            if nNumBatch % 100 == 0: #100개 배치마다 출력
                train_acc = sess.run(accr, feed_dict = feed_dictionary)
                print('\t[%d nNumBatch] train_cost = %g, acc = %g'%(nNumBatch, AvgBatchCost/nNumBatch, train_acc))
                loss_for_plot.append(AvgBatchCost/nNumBatch)
                acc_for_plot.append(train_acc)

        
    if epoch % 10 == 0: #10epoch마다 테스트데이터를 이용해 테스트
        save_path = saver.save(sess, SAVE_PATH+CHECKPOINT, global_step=epoch) #10epoch마다 저장
        print(save_path)
        test_acc = 0
        avg_acc = 0
        num_batch = 0
        load_testdata=load_npz(test_file[0]) #test를 위한 테스트에디터 로드
        
        testdata_label = one_hot_encode(load_testdata[b'labels'], N_CLASSES)
        test_batch_index = make_batch_index(load_testdata, BATCH_SIZE, False)

        for start, end in test_batch_index:
            num_batch += 1
            batch_xs = load_testdata[b'data'][start:end]
            batch_ys = one_hot_encode(load_testdata[b'labels'][start:end], N_CLASSES)
            test_acc = sess.run(accr, feed_dict={x:batch_xs,y:batch_ys})
            avg_acc += test_acc
            
        print('[%d epoch]test acc=%g'%(epoch, avg_acc/num_batch))
        summary_writer.add_summary(avg_acc/num_batch ,epoch)

	[100 nNumBatch] train_cost = 2.30357, acc = 0.09375
	[200 nNumBatch] train_cost = 2.3034, acc = 0.078125
	[300 nNumBatch] train_cost = 2.30349, acc = 0.078125
	[400 nNumBatch] train_cost = 2.30353, acc = 0.140625
	[500 nNumBatch] train_cost = 2.30354, acc = 0.125
	[600 nNumBatch] train_cost = 2.30413, acc = 0.09375
	[700 nNumBatch] train_cost = 2.30424, acc = 0.1875
checkpoint/simple_res_net/simple_res_net.ckpt-0
[0 epoch]test acc=0.10006
	[100 nNumBatch] train_cost = 2.30956, acc = 0.15625
	[200 nNumBatch] train_cost = 2.30755, acc = 0.078125
	[300 nNumBatch] train_cost = 2.30618, acc = 0.078125
	[400 nNumBatch] train_cost = 2.30557, acc = 0.140625
	[500 nNumBatch] train_cost = 2.30523, acc = 0.140625
	[600 nNumBatch] train_cost = 2.30499, acc = 0.1875
	[700 nNumBatch] train_cost = 2.30478, acc = 0.09375
	[100 nNumBatch] train_cost = 2.3038, acc = 0.078125
	[200 nNumBatch] train_cost = 2.3039, acc = 0.109375
	[300 nNumBatch] train_cost = 2.30355, acc = 0.140625
	[400 nNumBatch] train

	[400 nNumBatch] train_cost = 2.30359, acc = 0.078125
	[500 nNumBatch] train_cost = 2.30374, acc = 0.078125
	[600 nNumBatch] train_cost = 2.30379, acc = 0.078125
	[700 nNumBatch] train_cost = 2.30372, acc = 0.09375
	[100 nNumBatch] train_cost = 2.30366, acc = 0.078125
	[200 nNumBatch] train_cost = 2.30364, acc = 0.078125
	[300 nNumBatch] train_cost = 2.30349, acc = 0.015625
	[400 nNumBatch] train_cost = 2.30379, acc = 0.140625
	[500 nNumBatch] train_cost = 2.3038, acc = 0.125
	[600 nNumBatch] train_cost = 2.30371, acc = 0.09375
	[700 nNumBatch] train_cost = 2.3037, acc = 0.09375
	[100 nNumBatch] train_cost = 2.30348, acc = 0.15625
	[200 nNumBatch] train_cost = 2.3037, acc = 0.078125
	[300 nNumBatch] train_cost = 2.30354, acc = 0.015625
	[400 nNumBatch] train_cost = 2.30383, acc = 0.140625
	[500 nNumBatch] train_cost = 2.30383, acc = 0.125
	[600 nNumBatch] train_cost = 2.30373, acc = 0.09375
	[700 nNumBatch] train_cost = 2.30365, acc = 0.109375
	[100 nNumBatch] train_cost = 2.30375, acc

	[100 nNumBatch] train_cost = 2.30402, acc = 0.046875
	[200 nNumBatch] train_cost = 2.30396, acc = 0.0625
	[300 nNumBatch] train_cost = 2.3037, acc = 0.140625
	[400 nNumBatch] train_cost = 2.30373, acc = 0.125
	[500 nNumBatch] train_cost = 2.30371, acc = 0.09375
	[600 nNumBatch] train_cost = 2.30365, acc = 0.171875
	[700 nNumBatch] train_cost = 2.30364, acc = 0.0625
	[100 nNumBatch] train_cost = 2.30415, acc = 0.046875
	[200 nNumBatch] train_cost = 2.30395, acc = 0.078125
	[300 nNumBatch] train_cost = 2.30379, acc = 0.078125
	[400 nNumBatch] train_cost = 2.30383, acc = 0.125
	[500 nNumBatch] train_cost = 2.3038, acc = 0.078125
	[600 nNumBatch] train_cost = 2.30381, acc = 0.1875
	[700 nNumBatch] train_cost = 2.30378, acc = 0.09375
	[100 nNumBatch] train_cost = 2.30383, acc = 0.046875
	[200 nNumBatch] train_cost = 2.30369, acc = 0.109375
	[300 nNumBatch] train_cost = 2.30378, acc = 0.09375
	[400 nNumBatch] train_cost = 2.30358, acc = 0.078125
	[500 nNumBatch] train_cost = 2.30379, acc = 

	[600 nNumBatch] train_cost = 2.30264, acc = 0.078125
	[700 nNumBatch] train_cost = 2.30265, acc = 0.09375
	[100 nNumBatch] train_cost = 2.30263, acc = 0.078125
	[200 nNumBatch] train_cost = 2.30263, acc = 0.078125
	[300 nNumBatch] train_cost = 2.30265, acc = 0.09375
	[400 nNumBatch] train_cost = 2.30265, acc = 0.078125
	[500 nNumBatch] train_cost = 2.30264, acc = 0.109375
	[600 nNumBatch] train_cost = 2.30264, acc = 0.03125
	[700 nNumBatch] train_cost = 2.30265, acc = 0.171875
	[100 nNumBatch] train_cost = 2.30262, acc = 0.0625
	[200 nNumBatch] train_cost = 2.30263, acc = 0.078125
	[300 nNumBatch] train_cost = 2.30265, acc = 0.09375
	[400 nNumBatch] train_cost = 2.30265, acc = 0.140625
	[500 nNumBatch] train_cost = 2.30265, acc = 0.09375
	[600 nNumBatch] train_cost = 2.30264, acc = 0.109375
	[700 nNumBatch] train_cost = 2.30265, acc = 0.0625
	[100 nNumBatch] train_cost = 2.30262, acc = 0.0625
	[200 nNumBatch] train_cost = 2.30261, acc = 0.046875
	[300 nNumBatch] train_cost = 2.30263, 

	[400 nNumBatch] train_cost = 2.30264, acc = 0.0625
	[500 nNumBatch] train_cost = 2.30265, acc = 0.15625
	[600 nNumBatch] train_cost = 2.30265, acc = 0.03125
	[700 nNumBatch] train_cost = 2.30265, acc = 0.171875
	[100 nNumBatch] train_cost = 2.30264, acc = 0.109375
	[200 nNumBatch] train_cost = 2.30264, acc = 0.140625
	[300 nNumBatch] train_cost = 2.30263, acc = 0.09375
	[400 nNumBatch] train_cost = 2.30262, acc = 0.140625
	[500 nNumBatch] train_cost = 2.30263, acc = 0.125
	[600 nNumBatch] train_cost = 2.30265, acc = 0.078125
	[700 nNumBatch] train_cost = 2.30264, acc = 0.109375
	[100 nNumBatch] train_cost = 2.30265, acc = 0.0625
	[200 nNumBatch] train_cost = 2.30264, acc = 0.109375
	[300 nNumBatch] train_cost = 2.30264, acc = 0.078125
	[400 nNumBatch] train_cost = 2.30264, acc = 0.109375
	[500 nNumBatch] train_cost = 2.30265, acc = 0.140625
	[600 nNumBatch] train_cost = 2.30265, acc = 0.09375
	[700 nNumBatch] train_cost = 2.30265, acc = 0.09375
	[100 nNumBatch] train_cost = 2.30264, a

In [11]:
#loss, accracy graph
%matplotlib inline
from matplotlib import pyplot as plt

In [12]:
x = np.linspace(0, len(y), len(y))
y = loss_for_plot
plt.plot(x,y, color="red")
plt.plot()
plt.xlabel("batch")
plt.ylabel("loss")
plt.title("LOSS")




TypeError: object of type 'Tensor' has no len()

In [ ]:
x = np.linspace(0, len(y_ac), len(y_ac))
y_ac = acc_for_plot
plt.plot(x,y_ac, color="blue")
plt.plot()
plt.xlabel("batch")
plt.ylabel("accuracy")
plt.title("ACCURACY")

In [ ]:
sess.close()

In [ ]:
"""
남은것
1. augmentation - flip, blurr, sharp, crop..? 
2. data restore(save는 함) 111
3. 텐서보드(summary writer) https://www.tensorflow.org/get_started/summaries_and_tensorboard
4. visualization(filter weight값)
5. queue로 입력 받기
6. miss classification된 label, file_name 저장 > 나중에 확인
7. loss 파일 저장 > plot 1111
"""

In [ ]:
#CIFAR 데이터 읽어오기 
def read_dataset(filename_queue):
    class DataRecord(object):
        pass
    
    result = DataRecord()
    
    label_bytes = 1
    result.height = 32
    result.width = 32
    result.depth = 3
    image_bytes = result.height * result.width * result.depth
    record_bytes = label_bytes + image_bytes
    
    reader = tf.FixedLengthRecordReader(record_bytes = record_bytes)
    result.key, value = reader.read(filename_queue)
    print (value)
    record_bytes = tf.decode_raw(value, tf.unit8)
    
    result.label = tf.cast(tf.strided_slice(record_bytes, [0], [label_bytes]), tf.int32) #input, begin, end로 잘라냄 > 0~1
    
    depth_major = tf.reshape(tf.strided_slice(record_bytes, [label_bytes], # 1부터 image_bytes까지 image를 잘라낸 후 
                                              [label_bytes + image_bytes]), #channel x height x width로 변환함.
                             [result.depth, result.height, result.width])
    
    result.uint8image = tf.transpose(depth_major, [1,2,0]) #channel x height x width >>> height x width x chennl로 변경
    
    return result

In [ ]:
#label, image 세팅해줌 
def _generate_image_and_label_batch(image, label, min_queue_examples,
                                    batch_size, shuffle):
    """Construct a queued batch of images and labels.
    Args:
      image: 3-D Tensor of [height, width, 3] of type.float32.
      label: 1-D Tensor of type.int32
      min_queue_examples: int32, minimum number of samples to retain
        in the queue that provides of batches of examples.
      batch_size: Number of images per batch.
      shuffle: boolean indicating whether to use a shuffling queue.
    Returns:
      images: Images. 4D tensor of [batch_size, height, width, 3] size.
      labels: Labels. 1D tensor of [batch_size] size.
    """
    # Create a queue that shuffles the examples, and then
    # read 'batch_size' images + labels from the example queue.
    num_preprocess_threads = 16 #프로세스 스레드 
    if shuffle: #셔플 하면 
        images, label_batch = tf.train.shuffle_batch([image, label],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples + 3 * batch_size,
            min_after_dequeue=min_queue_examples) #이거만 해주면 batch와 동일함. 
    #Minimum number elements in the queue after a dequeue, used to ensure a level of mixing of elements.
    else:
        images, label_batch = tf.train.batch([image, label],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples + 3 * batch_size)

    # Display the training images in the visualizer.
    tf.summary.image('images', images) #걍 보드용인듯

    return images, tf.reshape(label_batch, [batch_size])


In [ ]:
def distorted_inputs(data_dir, batch_size): #좀 드럽게 인풋받아오기 
    """Construct distorted input for CIFAR training using the Reader ops.
    Args:
      data_dir: Path to the CIFAR-10 data directory.
      batch_size: Number of images per batch.
    Returns:
      images: Images. 4D tensor of [batch_size, IMAGE_SIZE, IMAGE_SIZE, 3] size.
      labels: Labels. 1D tensor of [batch_size] size.
    """
    filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i)
               for i in xrange(1, 6)] 
    for f in filenames:
        if not tf.gfile.Exists(f):
            raise ValueError('Failed to find file: ' + f)

    # Create a queue that produces the filenames to read.
    filename_queue = tf.train.string_input_producer(filenames)

    # Read examples from files in the filename queue.
    read_input = read_cifar10(filename_queue)
    reshaped_image = tf.cast(read_input.uint8image, tf.float32)

    height = IMAGE_SIZE
    width = IMAGE_SIZE

    # Image processing for training the network.  Note the many random
    # distortions applied to the image.

    # Randomly crop a [height, width] section of the image.
    distorted_image = tf.random_crop(reshaped_image, [height, width, 3])

    # Randomly flip the image horizontally.
    distorted_image = tf.image.random_flip_left_right(distorted_image)

    # Because these operations are not commutative, consider randomizing
    # the order their operation.
    distorted_image = tf.image.random_brightness(distorted_image,
                                               max_delta=63)
    distorted_image = tf.image.random_contrast(distorted_image,
                                             lower=0.2, upper=1.8)

    # Subtract off the mean and divide by the variance of the pixels.
    float_image = tf.image.per_image_standardization(distorted_image)

    # Set the shapes of tensors.
    float_image.set_shape([height, width, 3])
    read_input.label.set_shape([1])

    # Ensure that the random shuffling has good mixing properties.
    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN * min_fraction_of_examples_in_queue)
    print('Filling queue with %d CIFAR images before starting to train. '
         'This will take a few minutes.' % min_queue_examples)

    # Generate a batch of images and labels by building up a queue of examples.
    return _generate_image_and_label_batch(float_image, read_input.label,
                                         min_queue_examples, batch_size,
                                         shuffle=True)

In [ ]:
images, labels = cifar10.distorted_inputs()

In [13]:
Image to image translation

SyntaxError: invalid syntax (<ipython-input-13-60bcad50cbb0>, line 1)

In [14]:
Plug and play generative networks

SyntaxError: invalid syntax (<ipython-input-14-aab1e3f1db43>, line 1)

In [ ]:
Condtional gan 종류 dense captioning

Faster rcnn    squize       yolo     mask rcnn      sdd      squzeedet squeezeDETㅅ